# __Group Project 2 - Team Wilbur Atwater__
<br>


### __Team Members: Lara Gursoy (Project Manager), Kate O’Hara, Himalia Joshi, Qianhao Zhong, Bolor Erdene-Ochir, Moria Zhut__
<br>


## **1. Project Topic:**

> __Examining the minimum subsistence price for 3 different kinds of diets (keto, mediterranean, vegan) using one list of products, adjusting the function for each diet to put different constrictions on nutrition in accordance with each diet.__
<br>


## **2. Project Goal:**
> * __Find which diet is the most expensive and least expensive.__
> * __Find which diet is the least "nutritious" in comparison to federal nutrition recommendations and what its price is.__
> * __Compare our findings of each diet with preconceived notions/widespread opinions about each diet (eg: vegan is expensive/unhealthy).__
<br>


## **3. [A]Population of Interest:**
> __As Berkeley students ourselves, we wanted to see the minimum cost diets of possible college students that may be on these various diets. The main grocery stores around campus are Trader Joe’s, Safeway, and some small corner stores. Students most likely shop mostly at Trader Joe’s and Safeway as they have the most options and the more affordable prices.__

<br>


## **4. Project Deliverables:**

## Packages Import:

In [661]:
import pandas as pd
import numpy as np
import fooddatacentral as fdc
import warnings
from  scipy.optimize import linprog as lp
import cufflinks as cf

### [A] Dietary Reference Intakes:

In [674]:
gender = input("Gender(C or F or M): ").upper()
age_range = input("Age Range: ")  # Format of age_range: "M 31-50", "F 51+", "C 1-3"
max_or_min = input("Max or Min: ").lower()
min_requirement = pd.read_csv("Dietary_Requirements_diet_minimums.csv")
min_requirement = min_requirement.drop(min_requirement.columns[1], axis = 1)
max_requirement = pd.read_csv("Dietary_Requirements_diet_maximums.csv")
max_requirement = max_requirement.drop(max_requirement.columns[1], axis = 1)

def min_require(age_range, gender):  # return a pandas.Series
    query_char = str(gender) + " " + str(age_range)
    min_series = pd.Series(min_requirement[query_char].tolist(), index = min_requirement["Nutrition"].tolist())
    return min_series

def max_require(age_range, gender):  # return a pandas.Series
    query_char = str(gender) + " " + str(age_range)
    max_series = pd.Series(max_requirement[query_char].tolist(), index = max_requirement["Nutrition"].tolist())
    return max_series

if max_or_min == "min":
    print("\n")
    print(min_require(age_range, gender))
    print("\n")
    print(type(min_require(age_range, gender)))
elif max_or_min == "max":
    print("\n")
    print(max_require(age_range, gender))
    print("\n")
    print(type(max_require(age_range, gender)))

Gender(C or F or M): c
Age Range: 1-3
Max or Min: min


Energy                            1000.0
Protein                             13.0
Fiber, total dietary                14.0
Folate, DFE                        150.0
Calcium, Ca                        700.0
Carbohydrate, by difference        130.0
Iron, Fe                             7.0
Magnesium, Mg                       80.0
Niacin                               6.0
Phosphorus, P                      460.0
Potassium, K                      3000.0
Riboflavin                           0.5
Thiamin                              0.5
Vitamin A, RAE                     300.0
Vitamin B-12                         0.9
Vitamin B-6                          0.5
Vitamin C, total ascorbic acid      15.0
Vitamin E (alpha-tocopherol)         6.0
Vitamin K (phylloquinone)           30.0
Zinc, Zn                             3.0
dtype: float64


<class 'pandas.core.series.Series'>


### [A] Data on Prices for Difference foods:

In [675]:
food_price = pd.read_csv("Food_Price_Data.csv")
food_price = food_price.drop(food_price.columns[5:], axis = 1)
food_price

,fdcId,Name,Quantity,Unit,Price ($)
0,2120006,chicken breast tenders,1.00,lb,7.99
1,2390897,Atlantic salmon fillet,1.00,lb,10.99
2,2094567,Cage free large eggs,1.00,dozen,5.99
3,2344719,Avocado,5.00,oz,2.00
4,2577037,Extra virgin olive oil,17.00,fluid oz,7.99
5,2261421,Oat - Whole Grain,42.00,oz,6.99
6,2345841,Honey,12.00,oz,5.99
7,2067508,Greek Yogurt Plain,32.00,oz,6.99
8,2608592,Raw Almonds,16.00,oz,7.99
9,2012128,Banana,1.00,lb,1.99


In [676]:
# If it doesn't work, try to run the upper code block first!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

# Unit Conversion/Normalization
food_price["FDC Quantity"] = food_price.apply(lambda x: fdc.units(x["Quantity"], x["Unit"]), axis = 1)
food_price["FDC Price"] = food_price["Price ($)"] / food_price["FDC Quantity"]
warnings.filterwarnings("ignore", message="The unit of the quantity is stripped when downcasting to ndarray")
food_price = food_price.drop(food_price.columns[2:6], axis = 1)  
food_price

,fdcId,Name,FDC Price
0,2120006,chicken breast tenders,1.7614934748571718 / hectogram
1,2390897,Atlantic salmon fillet,2.4228802614118043 / hectogram
2,2094567,Cage free large eggs,nan / milliliter
3,2344719,Avocado,1.4109584779832163 / hectogram
4,2577037,Extra virgin olive oil,nan / milliliter
5,2261421,Oat - Whole Grain,0.5870595095894454 / hectogram
6,2345841,Honey,1.760758600649889 / hectogram
7,2067508,Greek Yogurt Plain,0.770515606336147 / hectogram
8,2608592,Raw Almonds,1.7614934748571718 / hectogram
9,2012128,Banana,0.4387199017479063 / hectogram


### [A] Nutritional Content of Different Foods:

In [677]:
# Get apikey from https://fdc.nal.usda.gov/
food = input("Food Name: ")  # "crunchy peanut butter"
apikey = "mF4KpJl1hVML9jWQcT8iJZYIRXjD67LVCFzrsqN4"
food_key_df = fdc.search(apikey, food)
food_key_df = food_key_df.drop(food_key_df.columns[3:], axis = 1)
food_key_df

Food Name: Avocado


,fdcId,description,commonNames
0,2345765,Avocado dressing,
1,2344719,"Avocado, raw",
2,173573,"Oil, avocado",
3,2345808,"Avocado, for use on a sandwich",
4,171706,"Avocados, raw, California",
5,171707,"Avocados, raw, Florida",
6,2344445,"Sushi roll, avocado",
7,171705,"Avocados, raw, all commercial varieties",
8,2345344,"Lettuce, salad with avocado, tomato, and/or ca...",
9,2425060,"AVOCADO CHUNKS, AVOCADO",NaN


In [678]:
FDCID = str(input("The fdcId you picked from above table: "))
nutrition_df = fdc.nutrients(apikey, FDCID)
nutrition_df

The fdcId you picked from above table: 2345765


,Quantity,Units
Protein,1.90,g
Total lipid (fat),43.30,g
"Carbohydrate, by difference",7.36,g
Energy,427.00,kcal
"Alcohol, ethyl",0.00,g
...,...,...
PUFA 20:5 n-3 (EPA),0.00,g
MUFA 22:1,0.00,g
PUFA 22:5 n-3 (DPA),0.00,g
"Fatty acids, total monounsaturated",9.42,g


### [A] Matrix $A$ Construction (Nutrients-Food):

In [679]:
def matrix_A_func(food_price):
    
    # Normalized units with corresponding prices
    p = food_price.drop(food_price.columns[0], axis = 1)
    p = p[~p['FDC Price'].astype(str).str.contains('nan', case=False, na=False)]
    # astype(str) converts 'FDC Price' column to strings and applies the str.contains method to filter out rows containing 'nan'. 

    p = p.set_index('Name')
    ###############################################################################################################################
    min_requirement = pd.read_csv("Dietary_Requirements_diet_minimums.csv")
    min_requirement = min_requirement.drop(min_requirement.columns[1], axis = 1)
    min_requirement = min_requirement.set_index('Nutrition')
    ###############################################################################################################################
    max_requirement = pd.read_csv("Dietary_Requirements_diet_maximums.csv")
    max_requirement = max_requirement.drop(max_requirement.columns[1], axis = 1)
    max_requirement = max_requirement.set_index('Nutrition')
    ###############################################################################################################################

    matrix_A = {}

    for food in food_price.Name.tolist():
        try:
            FDCID = food_price.loc[food_price.Name == food, :].fdcId.iloc[0]
            matrix_A[food] = fdc.nutrients(apikey, FDCID).Quantity   
        except AttributeError:
            warnings.warn(f"Couldn't find FDC Code {FDCID} for food {food}.")

    matrix_A = pd.DataFrame(matrix_A, dtype = float)
    matrix_A = matrix_A[p.index].fillna(0)

    # Drop the rows that we don't have constraints for.
    matrix_A_min = matrix_A.loc[min_requirement.index]
    matrix_A_max = matrix_A.loc[max_requirement.index]

    # Concatentate matrix_A_min and matrix_A_max
    matrix_A = pd.concat(([matrix_A_min, -matrix_A_max]))
    return matrix_A


matrix_A = matrix_A_func(food_price)
matrix_A

,chicken breast tenders,Atlantic salmon fillet,Avocado,Oat - Whole Grain,Honey,Greek Yogurt Plain,Raw Almonds,Banana,Spinach,Quinoa,...,Ground beef,Shredded cheese,Cheese Slices,Portabello Mushrooms,Tomatoes,Cucumber,Brussels Sprouts,Pork Loin Sirloin,Strawberries,Kidney(chicken)
Nutrition,,,,,,,,,,,,,,,,,,,,,
Energy,89.00,212.00,160.000,0.00000,304.000,88.00,567.0,312.00,21.00,107.00,...,0.00000,393.00,391.00,92.000,26.00,15.000,43.000,704.000,35.000,157.000
Protein,18.75,20.35,2.000,13.16875,0.300,4.71,20.0,12.50,1.67,2.86,...,17.53125,25.00,21.74,2.110,1.64,0.650,3.380,20.480,0.640,27.000
"Fiber, total dietary",0.00,0.00,6.700,10.50000,0.200,0.00,13.3,6.20,1.70,2.10,...,0.00000,0.00,0.00,1.300,1.60,0.500,3.800,0.000,1.800,0.000
"Folate, DFE",0.00,0.00,81.000,0.00000,2.000,0.00,0.0,0.00,0.00,0.00,...,0.00000,0.00,0.00,28.000,0.00,7.000,61.000,0.000,8.000,82.000
"Calcium, Ca",0.00,0.00,12.000,42.81000,6.000,135.00,200.0,125.00,33.00,14.00,...,6.88900,536.00,696.00,3.000,33.00,16.000,42.000,14.000,12.000,19.000
"Carbohydrate, by difference",0.00,0.00,8.530,69.91725,82.400,3.53,20.0,40.62,3.33,20.00,...,0.00000,3.57,4.35,3.870,4.92,3.630,8.950,0.000,7.630,0.000
"Iron, Fe",0.32,0.32,0.550,3.99700,0.420,0.11,3.6,1.12,1.50,0.77,...,1.96500,0.00,0.00,0.310,1.18,0.280,1.400,0.820,0.280,5.750
"Magnesium, Mg",0.00,0.00,29.000,125.30000,2.000,0.00,0.0,0.00,0.00,0.00,...,16.41000,0.00,0.00,0.000,0.00,13.000,23.000,23.000,12.000,12.000
Niacin,0.00,0.00,1.740,1.94500,0.121,0.00,0.0,0.00,0.00,0.00,...,0.00000,0.00,0.00,4.494,0.00,0.098,0.745,6.093,0.386,3.890


### [A] Matrix $b$ Construction (Nutrients-Age&Gender)

In [680]:
# Minimum Requirement DataFrame
min_requirement = pd.read_csv("Dietary_Requirements_diet_minimums.csv")
min_requirement = min_requirement.drop(min_requirement.columns[1], axis = 1)
min_requirement = min_requirement.set_index("Nutrition")
min_requirement

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


In [681]:
# Maximum Requirement DataFrame
max_requirement = pd.read_csv("Dietary_Requirements_diet_maximums.csv")
max_requirement = max_requirement.drop(max_requirement.columns[1], axis = 1)
max_requirement = max_requirement.set_index("Nutrition")
max_requirement

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
"Sodium, Na",1500,1900,1900,2200,2200,2300,2300,2300,2300,2300,2300,2300,2300
Energy,2500,2500,2500,2800,3000,3100,3100,3100,3100,3100,3100,3100,3100


In [682]:
# Concatenate min_requirement and max_requirement to get Matrix b
matrix_b = pd.concat([min_requirement, -max_requirement])
matrix_b

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


### [A] Objective Function ($p$)

In [683]:
# Normalized units with corresponding prices
p = food_price.drop(food_price.columns[0], axis = 1)
p = p[~p['FDC Price'].astype(str).str.contains('nan', case=False, na=False)]
# astype(str) converts 'FDC Price' column to strings and applies the str.contains method to filter out rows containing 'nan'. 

p = p.set_index('Name')
p_series = p["FDC Price"]
# str(p["FDC Price"].values[1])[:7]
list_p = []
for index in p_series:
    index = str(index)[:10]   # index is now string
    list_p.append(index)
    
p["FDC Price"] = list_p

p

,FDC Price
Name,
chicken breast tenders,1.76149347
Atlantic salmon fillet,2.42288026
Avocado,1.41095847
Oat - Whole Grain,0.58705950
Honey,1.76075860
Greek Yogurt Plain,0.77051560
Raw Almonds,1.76149347
Banana,0.43871990
Spinach,1.32056895


### [A] Solution:

In [684]:
# If this code doen't work, run from the top again!!!!!!!!!!!!
gender = input("Gender(C or F or M): ").upper()
age_range = input("Age Range: ")  # Format of age_range: "M 31-50", "F 51+", "C 1-3"
query_char_1 = str(gender) + " " + str(age_range)

tolerance_level = 1e-6
result = lp(p, -matrix_A, -matrix_b[query_char_1], method='highs')
result
print("\n")
print("Cost of diet for population group of %s is $%4.2f per day." % (query_char_1,result.fun))

print("\n")

diet_recommandation = pd.Series(result.x,index = p.index)
print("\nThey'll be eating (in hectograms or milliliters):")
print(diet_recommandation[diet_recommandation >= tolerance_level])

Gender(C or F or M): M
Age Range: 51+


Cost of diet for population group of M 51+ is $6.30 per day.



They'll be eating (in hectograms or milliliters):
Name
Green Cabbage        0.965633
Peanut Butter        1.360770
Carrots              9.264039
Whole Wheat Bread    2.419944
Long Grain rice      0.547805
Shredded cheese      0.328609
Kidney(chicken)      0.100172
dtype: float64


### [B] Is Your Solution Edible?
> __The diet below is the least cost diet we picked. The main ingredients include:__
> * __Oat - Whole Grain: &emsp;  &emsp; &emsp;0.02 hectograms__
> * __Green Cabbage: &nbsp;&emsp; &emsp; &emsp;&emsp;0.58 hectograms__
> * __Peanut Butter:   &emsp;&emsp;  &emsp;&emsp;&emsp;1.37 hectograms__
> * __Carrots: &emsp; &emsp; &ensp; &emsp; &emsp;  &emsp;   &ensp;   &nbsp;     9.13 hectograms__
> * __Whole Wheat Bread:     &emsp;  &emsp; 2.47 hectograms__
> * __Long Grain rice:  &emsp; &ensp;  &emsp;  &emsp;    1.27 hectograms__
> * __Shredded cheese:     &nbsp; &ensp;  &emsp; &emsp;  0.31 hectograms__
> * __Kidney(chicken):  &emsp; &nbsp;  &emsp;  &emsp;  0.10 hectograms__


> __Additional Ingredients:__
> * __Sesame Oil: &emsp;  &emsp; &emsp;1 tablespoon__
> * __Soy Sause: &nbsp; &emsp; &emsp;&emsp;2 tablespoons__


__This meal costs $6.34 per day.__

__The pictures of our meal below  [created by AI:](https://copilot.microsoft.com/)__

<br>


<div style="display: flex; justify-content: center; align-items: center;">
  <img src="food_1.png" alt="First Image" style="width: 45%; margin-right: 10px;">
  <img src="food_2.png" alt="Second Image" style="width: 45%;">
</div>

<br>


__The code block right below shows how we got the outcome:__

In [688]:
# If this code doen't work, run from the top again!!!!!!!!!!!!
tolerance_level = 1e-6
result = lp(p, -matrix_A, -matrix_b[query_char_1], method='highs')
result
print("\n")
print("The cost of %s is $%4.2f per day." % ("our diet",result.fun))

print("\n")

diet_recommandation = pd.Series(result.x,index = p.index)
print("\nWe'll be eating (in hectograms or milliliters):")
print(diet_recommandation[diet_recommandation >= tolerance_level])

print("\n")

tab = pd.DataFrame({"Outcome":np.abs(matrix_A).dot(diet_recommandation),"Recommendation":np.abs(matrix_b[query_char_1])})
print("\nWith the following nutritional outcomes of interest:")
tab



The cost of our diet is $6.30 per day.



We'll be eating (in hectograms or milliliters):
Name
Green Cabbage        0.965633
Peanut Butter        1.360770
Carrots              9.264039
Whole Wheat Bread    2.419944
Long Grain rice      0.547805
Shredded cheese      0.328609
Kidney(chicken)      0.100172
dtype: float64



With the following nutritional outcomes of interest:


,Outcome,Recommendation
Nutrition,,
Energy,2000.000000,2000.0
Protein,87.089146,56.0
"Fiber, total dietary",52.189347,28.0
"Folate, DFE",539.391061,400.0
"Calcium, Ca",1000.000000,1000.0
"Carbohydrate, by difference",274.514434,130.0
"Iron, Fe",14.886648,8.0
"Magnesium, Mg",586.073187,420.0
Niacin,46.144167,16.0


### [B] Meal Reviews:
> __By providing the csv file of a specific food list below, the following code will return the estimated price, estimated nutrients and the comparisions with *Recommended Dietary Allowances (RDAs)*.__

In [689]:
# If this code doen't work, run from the top again!!!!!!!!!!!!
Other_team_csv = str(input("Other team's diet csv file (e.g. Food_Price_Data.csv):"))
food_price_1 = pd.read_csv(Other_team_csv)  # It must be the same format as our "Food_Price_Data.csv"
food_price_1 = food_price_1.drop(food_price_1.columns[5:], axis = 1)
food_price_1["FDC Quantity"] = food_price_1.apply(lambda x: fdc.units(x["Quantity"], x["Unit"]), axis = 1)
food_price_1["FDC Price"] = food_price_1["Price ($)"] / food_price_1["FDC Quantity"]
warnings.filterwarnings("ignore", message="The unit of the quantity is stripped when downcasting to ndarray")
food_price_1 = food_price_1.drop(food_price_1.columns[2:6], axis = 1)  
print("\nThose are the ingredients in other team's diet:")
print(food_price_1)

print("\n")

matrix_A_1 = matrix_A_func(food_price_1)

tab_1 = pd.DataFrame({"Outcome":np.abs(matrix_A_1).dot(diet_recommandation),"Recommendation":np.abs(matrix_b["F 19-30"])})
print("\nWith the following nutritional outcomes of interest:")


result_1 = lp(p, -matrix_A_1, -matrix_b["F 19-30"], method='highs')

print("Cost of diet for the population group of %s is $%4.2f per day." % ("this team",result_1.fun))

print("\n")

tab_1

Other team's diet csv file (e.g. Food_Price_Data.csv):Food_Price_Data.csv

Those are the ingredients in other team's diet:
      fdcId                    Name                        FDC Price
0   2120006  chicken breast tenders   1.7614934748571718 / hectogram
1   2390897  Atlantic salmon fillet   2.4228802614118043 / hectogram
2   2094567    Cage free large eggs                 nan / milliliter
3   2344719                 Avocado   1.4109584779832163 / hectogram
4   2577037  Extra virgin olive oil                 nan / milliliter
5   2261421       Oat - Whole Grain   0.5870595095894454 / hectogram
6   2345841                   Honey    1.760758600649889 / hectogram
7   2067508      Greek Yogurt Plain    0.770515606336147 / hectogram
8   2608592             Raw Almonds   1.7614934748571718 / hectogram
9   2012128                  Banana   0.4387199017479063 / hectogram
10  2099349                 Spinach   1.3205689504874165 / hectogram
11  1858805                 Quinoa    0.835551973

,Outcome,Recommendation
Nutrition,,
Energy,2000.000000,2000.0
Protein,87.089146,46.0
"Fiber, total dietary",52.189347,28.0
"Folate, DFE",539.391061,400.0
"Calcium, Ca",1000.000000,1000.0
"Carbohydrate, by difference",274.514434,130.0
"Iron, Fe",14.886648,18.0
"Magnesium, Mg",586.073187,310.0
Niacin,46.144167,14.0


### [C] Sensitivity of Solution:

In [690]:
# Effects of Price Changes on Subsistence Diet Cost
cf.go_offline()
scale = [.5,.6,.7,.8,.9,1.,1.1,1.2,1.3,1.4,1.5]
cost0 = result.fun
Price_response={}

for s in scale:
    cost = {}
    for i in range(len(p)):
        my_p = p.copy()
        my_p['FDC Price'] = pd.to_numeric(my_p['FDC Price'], errors='coerce')
        my_p.iloc[i] = my_p.iloc[i] * s 
        result_2 = lp(my_p, -matrix_A, -matrix_b[query_char_1], method='highs')
        cost[p.index[i]] = np.log(result_2.fun / cost0)
    Price_response[np.log(s)] = cost

Price_response = pd.DataFrame(Price_response).T

layout = {
    'title': 'Effects of Price Changes on Subsistence Diet Cost',
    'xaxis': {'title': 'Change in Log Price'},
    'yaxis': {'title': 'Change in Log Cost'}
}

Price_response.iplot(layout=layout)

### [B] What is Total Cost for Population of Interest?

In [692]:
# We can also use this code block to analyze our project topic: Keto, Mediterranean and Vegan diets.
# If this code doen't work, run from the top again!!!!!!!!!!!!
Other_team_csv = str(input("Other team's diet csv file (e.g. Food_Price_Data.csv):"))
food_price_other_team = pd.read_csv(Other_team_csv)  # It must be the same format as our "Food_Price_Data.csv"
food_price_other_team = food_price_other_team.drop(food_price_other_team.columns[5:], axis = 1)
food_price_other_team["FDC Quantity"] = food_price_other_team.apply(lambda x: fdc.units(x["Quantity"], x["Unit"]), axis = 1)
food_price_other_team["FDC Price"] = food_price_other_team["Price ($)"] / food_price_other_team["FDC Quantity"]
warnings.filterwarnings("ignore", message="The unit of the quantity is stripped when downcasting to ndarray")
food_price_other_team = food_price_other_team.drop(food_price_other_team.columns[2:6], axis = 1)  
print("\nThose are the ingredients in other team's diet:")
print(food_price_other_team)

print("\n")

matrix_A_1 = matrix_A_func(food_price_other_team)

tab_1 = pd.DataFrame({"Outcome":np.abs(matrix_A_1).dot(diet_recommandation),"Recommendation":np.abs(matrix_b["F 19-30"])})
print("\nWith the following nutritional outcomes of interest:")


result_1 = lp(p, -matrix_A_1, -matrix_b["F 19-30"], method='highs')

print("Cost of diet for the population group of %s is $%4.2f per day." % ("this team",result_1.fun))

print("\n")

tab_1

Other team's diet csv file (e.g. Food_Price_Data.csv):Food_Price_Data.csv

Those are the ingredients in other team's diet:
      fdcId                    Name                        FDC Price
0   2120006  chicken breast tenders   1.7614934748571718 / hectogram
1   2390897  Atlantic salmon fillet   2.4228802614118043 / hectogram
2   2094567    Cage free large eggs                 nan / milliliter
3   2344719                 Avocado   1.4109584779832163 / hectogram
4   2577037  Extra virgin olive oil                 nan / milliliter
5   2261421       Oat - Whole Grain   0.5870595095894454 / hectogram
6   2345841                   Honey    1.760758600649889 / hectogram
7   2067508      Greek Yogurt Plain    0.770515606336147 / hectogram
8   2608592             Raw Almonds   1.7614934748571718 / hectogram
9   2012128                  Banana   0.4387199017479063 / hectogram
10  2099349                 Spinach   1.3205689504874165 / hectogram
11  1858805                 Quinoa    0.835551973

,Outcome,Recommendation
Nutrition,,
Energy,2000.000000,2000.0
Protein,87.089146,46.0
"Fiber, total dietary",52.189347,28.0
"Folate, DFE",539.391061,400.0
"Calcium, Ca",1000.000000,1000.0
"Carbohydrate, by difference",274.514434,130.0
"Iron, Fe",14.886648,18.0
"Magnesium, Mg",586.073187,310.0
Niacin,46.144167,14.0


<br>

### __<div style="text-align: right"> <span style="font-family:Papyrus; ">-------------------------- The End.</span></div>__